<a href="https://colab.research.google.com/github/1zpzz/QM2-Website-/blob/main/Data_cleaning_and_Visualisation_for_ESSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

Download xls files from United States Census Bereau, clean and combine them into a comprehensive datasets that contain PPSE data from 2007-2023.

In [ ]:
import requests
import pandas as pd
import os
from google.colab import files

def download_and_process_excel(url):
    filename = os.path.basename(url)
    response = requests.get(url)

    with open(filename, "wb") as file:
        file.write(response.content)


    df = pd.read_excel(
        filename,     # Use the correct filename
        sheet_name="20",
        skiprows=5,
        header=[0, 1]      # Use the next TWO rows as header rows
    )

    # Drop any completely empty rows or columns
    df.dropna(how="all", axis=0, inplace=True)
    df.dropna(how="all", axis=1, inplace=True)

    # Flatten the multi-level columns into a single level.
    df.columns = [
        "_".join(map(str, col)).strip()
        for col in df.columns
    ]

    #    Rename columns so they match desired format:
    #    - First column => "Geographic area"
    #    - "Percentage change1" => "PCT"
    def rename_col(col):
        if "Geographic area" in col:
            return "Geographic area"

        # Convert "Percentage change1" to "PCT"
        col = col.replace("Percentage change1", "PCT")

        return col

    df.columns = [rename_col(c) for c in df.columns]
    df['Geographic area'] = df['Geographic area'].str.rstrip('.')

    # remove the non-states data
    df = df.iloc[:52]

    return df


# download the data and save as dataframe
df1 = download_and_process_excel("https://www2.census.gov/programs-surveys/school-finances/tables/2022/secondary-education-finance/elsec22_sumtables.xls")
df2 = download_and_process_excel("https://www2.census.gov/programs-surveys/school-finances/tables/2017/secondary-education-finance/elsec17_sumtables.xls")
df3 = download_and_process_excel("https://www2.census.gov/programs-surveys/school-finances/tables/2012/secondary-education-finance/elsec12_sttables.xls")

PPCS data for 2023 is preliminary, the spreadsheet is in different format, hence slightly different data cleaning process is needed.

In [ ]:
def download_and_process_excel_2023(url):
    filename = os.path.basename(url)
    response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(response.content)

    df = pd.read_excel(
        filename,
        sheet_name="Table 1",
        skiprows=4,
        header=[0, 1]
    )

    # Drop empty rows/cols
    df.dropna(how="all", axis=0, inplace=True)
    df.dropna(how="all", axis=1, inplace=True)

    # Flatten multi-level columns
    df.columns = [
        "_".join(map(str, col)).strip()
        for col in df.columns
    ]

    # Rename function
    def rename_col(col):
        col_lower = col.lower()
        if "geographic area" in col_lower:
            return "Geographic area"
        elif col == "Elementary-secondary expenditure 1  _Per pupil":
            return "2023_PPCS"
        return col

    # Apply rename
    df.columns = [rename_col(c) for c in df.columns]

    # Clean up area names
    df["Geographic area"] = df["Geographic area"].str.rstrip(".")
    df["Geographic area"] = df["Geographic area"].str.rstrip("…")

    # Now keep just those two columns (if they exist)
    wanted_cols = ["Geographic area", "2023_PPCS"]
    df = df[wanted_cols]

    # Optionally drop extra rows
    df = df.iloc[:44]

    return df

df4 = download_and_process_excel_2023("https://www2.census.gov/programs-surveys/school-finances/tables/2023/secondary-education-finance/23elsec_prelim.xls")
df4

,Geographic area,2023_PPCS
0,NaN,current spending
1,NaN,(whole dollars)
3,Reporting Areas,15825.110117
4,Alaska,20339.725985
5,Arizona,11297.091877
6,Arkansas,12647.936785
7,California,18905.783463
9,Colorado,14863.59825
10,Connecticut,24895.87963
11,Delaware,21340.181592


clean the df4, to fit the data structure of other three df(df1, df2, df3), and complete the Geographic area column by adding missing states and fill the 2023_PPCS value for missing states with NaN.

In [ ]:
df4 = df4.iloc[3:].reset_index(drop=True)
df4

,Geographic area,2023_PPCS
0,Alaska,20339.725985
1,Arizona,11297.091877
2,Arkansas,12647.936785
3,California,18905.783463
4,Colorado,14863.59825
5,Connecticut,24895.87963
6,Delaware,21340.181592
7,Florida,11861.587117
8,Georgia,14413.636321
9,Hawaii,20055.655106


In [ ]:
import numpy as np
all_states = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado",
    "Connecticut","Delaware","District of Columbia","Florida","Georgia","Hawaii","Idaho",
    "Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana",
    "Maine","Maryland","Massachusetts","Michigan","Minnesota",
    "Mississippi","Missouri","Montana","Nebraska","Nevada",
    "New Hampshire","New Jersey","New Mexico","New York",
    "North Carolina","North Dakota","Ohio","Oklahoma","Oregon",
    "Pennsylvania","Rhode Island","South Carolina","South Dakota",
    "Tennessee","Texas","Utah","Vermont","Virginia","Washington",
    "West Virginia","Wisconsin","Wyoming"
]

states_df = pd.DataFrame(all_states, columns=["Geographic area"])

df4 = states_df.merge(df4, on="Geographic area", how="left")
df4.sort_values(by="Geographic area", inplace=True)
df4.reset_index(drop=True, inplace=True)
df4

,Geographic area,2023_PPCS
0,Alabama,NaN
1,Alaska,20339.725985
2,Arizona,11297.091877
3,Arkansas,12647.936785
4,California,18905.783463
5,Colorado,14863.59825
6,Connecticut,24895.87963
7,Delaware,21340.181592
8,District of Columbia,NaN
9,Florida,11861.587117


drop the pct data for df1, df2 and df3, leave only PPCS data and combine four dataframe into one.

In [ ]:
df1_ppcs = df1.filter(regex=r"(Geographic area|_PPCS$)").iloc[:, :-1]
df2_ppcs = df2.filter(regex=r"(Geographic area|_PPCS$)").iloc[:, :-1]
df3_ppcs = df3.filter(regex=r"(Geographic area|_PPCS$)")
for df in [df1, df2, df3, df4]:
    df["Geographic area"] = (
        df["Geographic area"]
        .str.strip()
    )
df_ppcs_2007_2023 = (
    df4
    .merge(df1_ppcs, on="Geographic area", how="inner")
    .merge(df2_ppcs, on="Geographic area", how="inner")
    .merge(df3_ppcs, on="Geographic area", how="inner")
)
df_ppcs_2007_2023['2023_PPCS'] = pd.to_numeric(df_ppcs_2007_2023['2023_PPCS'], errors='coerce')
numeric_cols = df_ppcs_2007_2023.select_dtypes(include=['number']).columns # find the numerical colummns
df_ppcs_2007_2023[numeric_cols] = df_ppcs_2007_2023[numeric_cols].round(2) # round to 2dp
df_ppcs_2007_2023

,Geographic area,2023_PPCS,2022_PPCS,2021_PPCS,2020_PPCS,2019_PPCS,2018_PPCS,2017_PPCS,2016_PPCS,2015_PPCS,2014_PPCS,2013_PPCS,2012_PPCS,2011_PPCS,2010_PPCS,2009_PPCS,2008_PPCS,2007_PPCS
0,Alabama,NaN,11819.43,10683.12,10116.29,10076.59,9696.18,9510.74,9242.68,9127.93,9027.69,8755.23,8562.06,8812.70,8880.74,8870.00,9103.36,8390.62
1,Alaska,20339.73,20190.93,19540.42,18313.42,18393.57,17725.94,17837.86,17509.98,20172.49,18415.91,18175.00,17390.40,16673.96,15782.50,15551.82,14629.71,12300.20
2,Arizona,11297.09,10314.74,9611.16,8784.97,8625.47,8254.80,8004.12,7613.01,7489.50,7527.77,7207.75,7558.92,7665.84,7848.08,7813.27,7607.74,7196.30
3,Arkansas,12647.94,12159.10,11266.29,10344.88,10387.82,10138.68,9966.56,9845.57,9693.80,9615.73,9394.45,9410.62,9353.38,9143.21,8711.92,8541.25,8283.57
4,California,18905.78,17049.22,14985.25,14031.30,14034.03,12142.72,12144.78,11495.36,10466.53,9594.76,9219.51,9182.89,9148.75,9374.71,9657.49,9863.39,9152.39
5,Colorado,14863.60,13422.07,12875.72,11602.36,11029.67,10201.96,9808.60,9574.74,9245.03,8985.19,8646.67,8547.66,8723.76,8852.78,8718.48,9078.58,8166.77
6,Connecticut,24895.88,24452.96,22769.26,21345.60,21310.45,20634.92,19321.72,18957.84,18377.29,17744.72,16631.12,16273.65,15616.50,14906.37,14531.12,13848.00,12979.33
7,Delaware,21340.18,19356.75,17447.59,17234.54,16314.69,15638.63,15301.68,14713.37,14120.43,13937.67,13832.89,13864.57,12684.98,12382.70,12257.22,12253.17,11828.96
8,District of Columbia,NaN,27424.71,24535.11,22855.92,22561.58,22758.86,21974.00,20235.00,19395.65,18485.05,19076.19,17468.47,18475.08,18666.85,16407.68,14594.34,14324.41
9,Florida,11861.59,11075.89,10401.22,9936.72,9645.31,9346.04,9075.49,8919.96,8881.14,8755.37,8432.60,8371.97,8886.54,8741.32,8760.38,9034.82,8513.77


Run the cell below to download the csv file for the dataset.

In [ ]:
df_ppcs_2007_2023.to_csv("df_ppcs_2007_2023.csv", index=False)
files.download("df_ppcs_2007_2023.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualisation

reformat the dataframe so it work with plotly.

In [ ]:
STATE_TO_ABBREV = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}
df_long = df_ppcs_2007_2023.melt(
    id_vars="Geographic area",
    var_name="year",
    value_name="PPCS"
)

df_long["year"] = df_long["year"].str.replace("_PPCS", "")
df_long["state_code"] = df_long["Geographic area"].map(STATE_TO_ABBREV)
df_long["PPCS "] = df_long["PPCS"].apply(lambda x: '${:.2f}'.format(x))

Interactive map for PPCS df

In [ ]:
import plotly.express as px
ordered_years = [str(y) for y in range(2007, 2024)]
fig = px.choropleth(
    data_frame=df_long,
    locations="state_code",
    locationmode="USA-states",
    color="PPCS",
    hover_name="Geographic area",
    hover_data={"PPCS ":True,"PPCS":False,"state_code":False,"year":False},
    animation_frame="year",
    category_orders={"year": ordered_years},
    scope="usa",
    color_continuous_scale="Blues",
)


fig.update_layout(
    title_text="Per Pupil Amounts for Current Spending of Public k-12 School Systems",
    geo=dict(lakecolor="white"),
    margin={"r":0, "t":50, "l":0, "b":0}
)
for f in fig.frames:
    if f.name == "2015":
        f.layout.annotations = [
            dict(
                x=1.01,
                y=0.8,
                xref="paper",
                yref="paper",
                text=(
                    "The Every Student Succeeds Act (ESSA) was signed into law <br>"
                    "into law by President Barack Obama on December 10, 2015, <br>"
                    "the law did not go into effect until 2017."
                ),
                font=dict(size=8),
                showarrow=False,
            )
        ]
fig.show()

Run the cell below to download the html files for the interactive map

In [ ]:
fig.write_html("ppcs_map.html")
files.download('ppcs_map.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>